<a href="https://colab.research.google.com/github/Kiyarash00/KD_Empire/blob/main/US_Accidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 25.08 libraries
Using Python 3.12.11 environment at: /usr
Audited 9 packages in 137ms

        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files.
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        ***********************************************************************
        


In [ ]:
import cuml
cuml.__version__

'25.08.00'

In [ ]:
import cudf
cuml.__version__

'25.08.00'

In [ ]:
!pip install kaggle --quiet

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
us-accidents.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip us-accidents.zip

Archive:  us-accidents.zip
replace US_Accidents_March23.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#This Cell Was for The Time I Couldn't establish Connection With GPU
#import dask.dataframe as dd

#df = dd.read_csv("/content/US_Accidents_March23.csv")

In [ ]:
#df = df.compute()

In [ ]:
df = cudf.read_csv("/content/US_Accidents_March23.csv")

In [ ]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   Severity               int64
 3   Start_Time             object
 4   End_Time               object
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object
 11  Street                 object
 12  City                   object
 13  County                 object
 14  State                  object
 15  Zipcode                object
 16  Country                object
 17  Timezone               object
 18  Airport_Code           object
 19  Weather_Timestamp      object
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)            float64
 23  

In [ ]:
df["Severity"] = df["Severity"].astype("uint8")

floats = df.select_dtypes(include=["float64"]).columns
df[floats] = df[floats].astype("float32")

bools = df.select_dtypes(include=["bool"]).columns
df[bools] = df[bools].astype("uint8")

cols_to_drop = [
    "ID", "Source", "County", "Start_Time", "End_Time",
    "Start_Lat", "Start_Lng", "End_Lat", "End_Lng",
    "Street", "Country", "City", "Zipcode",
    "Sunrise_Sunset", "Civil_Twilight", "Nautical_Twilight", "Astronomical_Twilight",
    "Timezone", "Airport_Code", "Weather_Timestamp",
    "Description"
]
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

state = df["State"].to_numpy()
wind_direction = df["Wind_Direction"].to_numpy()
weather_condition = df["Weather_Condition"].to_numpy()

le_state = LabelEncoder()
le_direction = LabelEncoder()
le_weather = LabelEncoder()

state_encoded = le_state.fit_transform(state)
wind_direction_encoded = le_direction.fit_transform(wind_direction)
weather_condition_encoded = le_weather.fit_transform(weather_condition)

df["State_Encoded"] = cudf.Series(state_encoded)
df["Wind_Direction_Encoded"] = cudf.Series(wind_direction_encoded)
df["Weather_Condition_Encoded"] = cudf.Series(weather_condition_encoded)

df.drop(columns=["State","Wind_Direction","Weather_Condition"], inplace=True)

In [ ]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 25 columns):
 #   Column                     Dtype
---  ------                     -----
 0   Severity                   uint8
 1   Distance(mi)               float32
 2   Temperature(F)             float32
 3   Wind_Chill(F)              float32
 4   Humidity(%)                float32
 5   Pressure(in)               float32
 6   Visibility(mi)             float32
 7   Wind_Speed(mph)            float32
 8   Precipitation(in)          float32
 9   Amenity                    uint8
 10  Bump                       uint8
 11  Crossing                   uint8
 12  Give_Way                   uint8
 13  Junction                   uint8
 14  No_Exit                    uint8
 15  Railway                    uint8
 16  Roundabout                 uint8
 17  Station                    uint8
 18  Stop                       uint8
 19  Traffic_Calming            uint8
 20  Traffic_Signal             u

In [ ]:
from xgboost import XGBClassifier

X = df[["Distance(mi)", "Temperature(F)", "Wind_Chill(F)","Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)", "Wind_Direction_Encoded","Precipitation(in)","Weather_Condition_Encoded", "State_Encoded"]]
y = df["Severity"] - 1

clf = XGBClassifier(
    device = "cuda",
    learning_rate=0.001,
    n_estimators=800,
    max_depth=9,
)

clf.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.001, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=800,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X)

print(classification_report(y.to_numpy(), y_pred))

              precision    recall  f1-score   support

           0       0.81      0.00      0.00     67366
           1       0.82      0.98      0.89   6156981
           2       0.59      0.14      0.23   1299337
           3       0.54      0.01      0.02    204710

    accuracy                           0.81   7728394
   macro avg       0.69      0.28      0.29   7728394
weighted avg       0.77      0.81      0.75   7728394



In [ ]:
del df
del clf